In [3]:
import os
import sys
import cv2
import json
import shutil
from glob import glob
from tqdm import tqdm
from pathlib import Path

from utils.table2label import table2layout, fuse_gt_info, judge_error
from utils.table_helper import correct_table



def table2label(table_dir, label_dir, error_file_path):
    table_error = {}
    json_files = sorted(glob(os.path.join(table_dir, '*.json')))
    for idx, json_path in tqdm(enumerate(json_files), total=len(json_files)):
        json_dir = os.path.dirname(json_path)
        json_name = os.path.basename(json_path)
        # json_name = "06615.json"
        # json_path = os.path.join(json_dir, json_name)

        json_id = json_name.split('.')[0]
        table = json.load(open(json_path, 'r'))

        table['is_wireless'] = True

        # table = correct_table(table)
        try:
            gt_label = table2layout(table)
        except:
            table_error[json_id] = 'table2layout error'
            continue

        ## 有线表格得到的bbox还是cell框，不是text框
        try:
            gt_label = fuse_gt_info(gt_label, table)
        except:
            table_error[json_id] = "fuse_gt_info error" # 仅有1cell的有线表格 滤过
            continue

        valid, msg = judge_error(table, gt_label)
        if not valid:
            print(json_name, msg)
            table_error[json_id] = msg
            continue

        gt_json_path = os.path.join(label_dir, f'{json_id}-gt.json')
        json.dump(gt_label, open(gt_json_path, 'w'), indent=4)
    json.dump(table_error, open(error_file_path, 'w'), indent=4)

    print('table error: {}'.format(len(table_error)))

## STEP.1 gen_gt_labels

In [4]:
'''
输入
    训练集目录: {dataset_root}/train
输出
    训练集标注目录: {dataset_root}/train_gt_json/
    {dataset_root}/train_error.json

使用 fix_gt_table 前 报错的：684
使用 fix_gt_table 后 报错的：665
    + 清除area==0的数据：664
'''
DATASET = "train_jpg480max"
DATASET_ROOT = '/media/ubuntu/Date12/TableStruct/new_data'

TABLE_DIR = os.path.join(DATASET_ROOT, DATASET)
LABEL_DIR = os.path.join(DATASET_ROOT, f'{DATASET}_gt_json')
TABLE_ERROR_PATH = os.path.join(DATASET_ROOT, f'{DATASET}_error.json')

if os.path.exists(LABEL_DIR):
    shutil.rmtree(LABEL_DIR)
Path.mkdir(Path(LABEL_DIR), parents=True)

table2label(TABLE_DIR, LABEL_DIR, TABLE_ERROR_PATH)


  0%|          | 36/12104 [00:01<08:36, 23.36it/s]

00029.json line idx:[[6], [12], [113]] not find


  1%|          | 61/12104 [00:02<09:48, 20.45it/s]

00057.json line idx:[[34]] not find


  1%|          | 92/12104 [00:03<05:57, 33.56it/s]

...

11882.json line idx:[[0]] not find


 99%|█████████▉| 12009/12104 [05:01<00:03, 26.10it/s]

12004.json line idx:[[57]] not find
12006.json line idx:[[275]] not find
12009.json line idx:[[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12], [13], [14], [15]] not find


100%|██████████| 12104/12104 [05:04<00:00, 39.76it/s]

12099.json line idx:[[3], [6], [8]] not find
table error: 280
